In [1]:
import pandas as pd

In [2]:
# Load data
df_gdp_mainlandchina = pd.read_csv('raw_data/data-gdp-mainlandchina-2019.csv')  
df_gdp_usa = pd.read_csv('raw_data/data-gdp-usa-2019.csv')  
df_pop_china = pd.read_csv('raw_data/data-pop-china-2017.csv')  
df_pop_usa = pd.read_csv('raw_data/data-pop-usa-2019.csv')  
df_WEO = pd.read_csv('raw_data/WEO_Data.csv')  

# China

In [3]:
# Merge dataframe for China
df_china = pd.DataFrame.merge(df_gdp_mainlandchina,df_pop_china,how='right',left_on='Provinces',right_on='Administrative Division')

# Don't show Unnamed columns
df_china = df_china.loc[:, ~df_china.columns.str.contains('^Unnamed')] 

# Drop column
df_china = df_china.drop(['Provinces'], axis=1)

# Add column
df_china['Country'] = 'China'
df_china['Year'] = '2019'

# Re-order Columns
df_china = df_china[['Year','Country','Administrative Division','Nominal GDP (Billion)','Population']]

df_china

,Year,Country,Administrative Division,Nominal GDP (Billion),Population
0,2019,China,Guangdong,"1,560.79","111,690,000"
1,2019,China,Jiangsu,"1,444.25","80,290,000"
2,2019,China,Shandong,"1,030.19","100,060,000"
3,2019,China,Zhejiang,903.84,"56,570,000"
4,2019,China,Henan,786.54,"95,590,000"
5,2019,China,Sichuan,675.74,"83,020,000"
6,2019,China,Hubei,664.32,"59,020,000"
7,2019,China,Fujian,614.55,"39,110,000"
8,2019,China,Hunan,576.24,"68,600,000"
9,2019,China,Shanghai,553.1,"24,180,000"


In [4]:
# Review data
df_WEO.head()

,Country,Subject Descriptor,Units,Scale,2017,2018,2019,2020,2021,2022,2023,2024,Estimates Start After
0,Afghanistan,"Gross domestic product, current prices",U.S. dollars,Billions,20.235,19.63,18.734,18.861,19.998,21.54,23.237,25.19,2018.0
1,Afghanistan,"Gross domestic product per capita, current prices",U.S. dollars,Units,569.531,544.983,513.108,509.759,533.089,566.416,602.884,644.95,2016.0
2,Afghanistan,"Inflation, average consumer prices",Index,NaN,110.998,111.693,114.583,119.742,125.427,131.699,138.284,145.198,2018.0
3,Afghanistan,Unemployment rate,Percent of total labor force,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,Population,Persons,Millions,35.53,36.02,36.51,37,37.514,38.028,38.543,39.057,2016.0


In [5]:
# Get GDP value for Hong Kong, Macau and Taiwan
df_hongkong = df_WEO[df_WEO['Country'].str.contains('hong kong', case=False, na=False)]
hongkong_gdp = df_hongkong.loc[df_hongkong['Subject Descriptor'] == 'Gross domestic product, current prices', '2019'].values[0]

df_macau = df_WEO[df_WEO['Country'].str.contains('maca', case=False, na=False)]
macau_gdp = df_macau.loc[df_macau['Subject Descriptor'] == 'Gross domestic product, current prices', '2019'].values[0]

df_taiwan = df_WEO[df_WEO['Country'].str.contains('taiwan', case=False, na=False)]
taiwan_gdp = df_taiwan.loc[df_taiwan['Subject Descriptor'] == 'Gross domestic product, current prices', '2019'].values[0]

print('Hong Kong GDP: ',hongkong_gdp)
print('Macau GDP: ',macau_gdp)
print('Taiwan GDP: ',taiwan_gdp)

Hong Kong GDP:  372.989
Macau GDP:  55.136
Taiwan GDP:  586.104


In [6]:
# Assign value
## to Hong Kong
hongkong_index = df_china.loc[df_china['Administrative Division'] == 'Hong Kong'].index[0]
df_china.at[hongkong_index, 'Nominal GDP (Billion)'] = hongkong_gdp

## to Macau
macau_index = df_china.loc[df_china['Administrative Division'] == 'Macau'].index[0]
df_china.at[macau_index, 'Nominal GDP (Billion)'] = macau_gdp

## to Taiwan 
taiwan_index = df_china.loc[df_china['Administrative Division'] == 'Taiwan'].index[0]
df_china.at[taiwan_index, 'Nominal GDP (Billion)'] = taiwan_gdp

df_china

,Year,Country,Administrative Division,Nominal GDP (Billion),Population
0,2019,China,Guangdong,"1,560.79","111,690,000"
1,2019,China,Jiangsu,"1,444.25","80,290,000"
2,2019,China,Shandong,"1,030.19","100,060,000"
3,2019,China,Zhejiang,903.84,"56,570,000"
4,2019,China,Henan,786.54,"95,590,000"
5,2019,China,Sichuan,675.74,"83,020,000"
6,2019,China,Hubei,664.32,"59,020,000"
7,2019,China,Fujian,614.55,"39,110,000"
8,2019,China,Hunan,576.24,"68,600,000"
9,2019,China,Shanghai,553.1,"24,180,000"


# USA

In [7]:
df_gdp_usa.head()

,State,Nominal GDP (Billion)
0,Connecticut,"1,142,561"
1,Maine,"270,074"
2,Massachusetts,"2,382,235"
3,New Hampshire,"354,379"
4,Rhode Island,"254,164"


In [8]:
df_pop_usa.head()

,State,Population
0,California,"39,512,223"
1,Texas,"28,995,881"
2,Florida,"21,477,737"
3,New York,"19,453,561"
4,Pennsylvania,"12,801,989"


In [9]:
# Strip leading and trailng white space in State column
df_gdp_usa['State'] = df_gdp_usa['State'].str.strip()
df_pop_usa['State'] = df_pop_usa['State'].str.strip()

# Merge dataframe for China
df_usa = pd.DataFrame.merge(df_gdp_usa,df_pop_usa,how='left',on='State')

# Add column
df_usa['Country'] = 'United States'
df_usa['Year'] = '2019'

# Rename column
df_usa.rename(columns={'State':'Administrative Division'}, inplace=True)

# Re-order Columns
df_usa = df_usa[['Year','Country','Administrative Division','Nominal GDP (Billion)','Population']]

df_usa

,Year,Country,Administrative Division,Nominal GDP (Billion),Population
0,2019,United States,Connecticut,"1,142,561","3,565,287"
1,2019,United States,Maine,"270,074","1,344,212"
2,2019,United States,Massachusetts,"2,382,235","6,949,503"
3,2019,United States,New Hampshire,"354,379","1,359,711"
4,2019,United States,Rhode Island,"254,164","1,059,361"
5,2019,United States,Vermont,"139,140","623,989"
6,2019,United States,Delaware,"301,661","973,764"
7,2019,United States,District of Columbia,"584,778","705,749"
8,2019,United States,Maryland,"1,713,354","6,045,680"
9,2019,United States,New Jersey,"2,579,366","8,882,190"


# Concat

In [10]:
# Concat data frame
clean_df = pd.concat([df_china, df_usa])

# Reset index
clean_df.reset_index(inplace = True)

clean_df

,index,Year,Country,Administrative Division,Nominal GDP (Billion),Population
0,0,2019,China,Guangdong,"1,560.79","111,690,000"
1,1,2019,China,Jiangsu,"1,444.25","80,290,000"
2,2,2019,China,Shandong,"1,030.19","100,060,000"
3,3,2019,China,Zhejiang,903.84,"56,570,000"
4,4,2019,China,Henan,786.54,"95,590,000"
5,5,2019,China,Sichuan,675.74,"83,020,000"
6,6,2019,China,Hubei,664.32,"59,020,000"
7,7,2019,China,Fujian,614.55,"39,110,000"
8,8,2019,China,Hunan,576.24,"68,600,000"
9,9,2019,China,Shanghai,553.1,"24,180,000"


In [11]:
clean_df.to_csv('clean_data/clean_china_and_usa.csv')